- Source: https://tableau.github.io/server-client-python/docs/api-ref
- "HDC현대산업개발": "2a21240d-3f53-4f02-944d-b01647c3dd51"

In [11]:
import tableauserverclient as TSC
import pandas as pd
import pickle as pk

pd.set_option("display.max_rows", None)

In [24]:
hr = pd.read_excel("D:/디지털혁신팀/태블로/직원명단_조직_210630.xlsx")
hr["성명"] = hr["성명"].apply(lambda x:x.replace(" ", ""))
hr = hr[["사번", "본부명(조직단위기준)", "조직명", "직무", "성명"]]
hr = hr.rename({"사번":"num", "본부명(조직단위기준)":"hq", "조직명":"dep", "직무":"job", "성명":"name"}, axis=1)
hr["num"] = hr["num"].map(str)

In [26]:
empls = {str(row["name"]):(row["hq"], row["dep"], row["job"], row["name"]) for _, row in hr.iterrows()}

req_opts = TSC.RequestOptions(pagesize=1000)

## SIgn In

- job: "건축PM", "단장", "소장"(일부 누락), "사업소장"(결정 필요), "팀장", "TFT팀장", "실장"(결정 필요)

In [4]:
with open("D:/디지털혁신팀/태블로/admin_TOKEN.pkl", "rb") as f:
    token = pk.load(f)
tableau_auth = TSC.PersonalAccessTokenAuth("admin_TOKEN", token)
server = TSC.Server("http://218.153.56.75/", use_server_version=True)

In [5]:
with server.auth.sign_in(tableau_auth):
    all_users, pagination_item = server.users.get(req_options=req_opts)
all_users = [(user.id, user.name, user.fullname, user.site_role) + empls[user.name] if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None) for user in all_users]
all_users = pd.DataFrame(all_users, columns=["user_id", "user_name", "user_fullname", "site_role", "hq", "dep", "job", "name"])
# all_users = all_users.sort_values(by=["hq", "dep", "job", "name", "site_role"])
# all_users = all_users.reset_index(drop=True)

In [6]:
# all_users.to_csv("D:/디지털혁신팀/태블로/all_users.csv", index=False, encoding="euc-kr")
all_users.head()

,user_id,user_name,user_fullname,site_role,hq,dep,job,name
0,bdebde0b-b41d-4e9a-a6b7-423ab68bb16b,admin,admin,ServerAdministrator,None,None,None,None
1,a6e879b8-950e-4b05-9ff6-fe5565da5018,viewer,viewer(디지털플랫폼팀),Viewer,None,None,None,None
2,2c2ceaa8-9387-4edb-8ab2-38b17f686304,2912,김동진,Viewer,건설본부,건축PM,건축PM,김동진
3,c232b3be-3aac-4f4f-83fb-bc8a15dab963,3109,배영선,Viewer,건설본부,건축PM,건축PM,배영선
4,e17666e4-a736-40a7-badc-865a60ebd7f2,3139,서태철,Viewer,건설본부,광주학동4구역,소장,서태철


In [27]:
hr["account"] = hr.apply(lambda x:True if x["num"] in all_users["user_name"].tolist() else False, axis=1)

In [33]:
hr.to_csv("D:/디지털혁신팀/태블로/hr.csv", encoding="euc-kr")

In [28]:
hr["job"].value_counts()

팀원            229
공구장           198
건축            193
안전            149
시험             72
토목             70
원가관리자          66
소장             64
도시재생           54
서무             50
A/S            48
보건관리자          45
출납             38
팀장             34
자재             32
PAM            20
기전공            18
전기             17
총무             16
기계             14
건축설계           14
영업             13
관리             13
아이서비스          10
설계             10
BS&C            9
센터장             9
전산              9
개발              8
원가보조            8
인프라             8
조경              7
분양사무원           7
총노무             7
건축토목            7
비서              7
철거담당자           7
사업소장            7
리모델링            6
신규준비            6
법무              6
계약직자문           6
기획              6
회계/자산관리         6
도로,철도           5
그룹장             5
휴직              5
감사              5
건축PM            4
본부장             4
설비              4
부문장             4
사외이사            4
민수사업            4
언론              4
사업        

In [29]:
hr[hr["job"]=="팀장"]

,num,hq,dep,job,name,account
51,3449,건설본부,외주구매팀,팀장,정상민,True
112,3881,개발영업본부,용현학익1블록도시개발사업단,팀장,장수호,True
127,3974,건설본부,건설디지털팀,팀장,강민석,True
154,4114,건설본부,공정상생팀,팀장,박혁종,True
155,4116,개발영업본부,도시정비1팀,팀장,문치성,True
161,4154,건설본부,건설기획팀,팀장,박정화,True
245,4587,건설본부,실시설계팀,팀장,최창구,True
253,4610,건설본부,인프라팀,팀장,김용주,True
255,4628,개발영업본부,용현학익1블록도시개발사업단,팀장,심재병,True
279,4785,건설본부,일반견적팀,팀장,이유로,True


## Projects

In [48]:
with server.auth.sign_in(tableau_auth):
    all_project_items, pagination_item = server.projects.get(req_options=req_opts)
    print([(prj.name, prj.id) for prj in all_project_items])

[('기본값', '7ff34228-4f13-11eb-8d94-cbbd9e107787'), ('Tableau 샘플', '07b1c1a3-d2fb-4a12-b4ba-c65b396f593c'), ('HDC현대산업개발', '2a21240d-3f53-4f02-944d-b01647c3dd51'), ('6월 데이터 확인', '44546321-94c8-4746-b0e5-0dfb3fa4ce5a')]


## Workbooks

In [307]:
# with server.auth.sign_in(tableau_auth):
#     all_wbs, pagination_item = server.workbooks.get()
# all_wbs = [(wb.project_name, wb.name) for wb in all_wbs]
# all_wbs = pd.DataFrame(all_wbs, columns=["prj", "wb"])
# all_wbs = all_wbs.sort_values(by=["prj", "wb"])

## Groups

## groups.get(), groups.populate_users()

In [34]:
with server.auth.sign_in(tableau_auth):
    all_groups, pagination_item = server.groups.get(req_options=req_opts)
all_groups = [(group.name, group.id) for group in all_groups]
all_groups = pd.DataFrame(all_groups, columns=["group", "group_id"])
# all_groups = all_groups.sort_values(by=["group", "group_id"])
# all_groups = all_groups.reset_index(drop=True)

In [162]:
all_groups.to_csv("D:/디지털혁신팀/태블로/all_groups.csv", index=False, encoding="euc-kr")
all_groups.head()

,group,group_id
0,"""HR현황"" 접근 가능 그룹",8bae0a19-c83c-4713-b932-a489754653de
1,"""본부장 회의"" 접근 가능 그룹",01b579e9-e0b0-458d-b12a-94292171d3a7
2,"""분양일보"" 접근 가능 그룹",16065e45-21b3-4e1b-80e1-c7c41079adfd
3,All Users,80307210-4f13-11eb-8d95-5bc411fd9c23
4,디지털혁신팀,516aa22e-b1ef-47b1-ad64-6e5116622c7f


In [82]:
group_user = list()
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    for group in groups[1:]:
        pagination_item = server.groups.populate_users(group, req_options=req_opts)
        group_user.extend([(group.name,) + empls[user.name] + (user.site_role,) if user.name in empls.keys() else (group.name, None, None, None, user.name, user.site_role) for user in group.users])

group_user = pd.DataFrame(group_user, columns=["group", "hq", "dep", "job", "name", "site_role"])
group_user = group_user.sort_values(by=["group", "hq", "dep", "job", "name", "site_role"])
group_user = group_user.reset_index(drop=True)

In [83]:
group_user.to_csv("D:/디지털혁신팀/태블로/group_user.csv", index=False, encoding="euc-kr")
group_user.head()

,group,hq,dep,job,name,site_role
0,HR현황,건설본부,건설기획팀,팀원,심창식,Viewer
1,HR현황,미래혁신본부,HR혁신팀,팀장,선영욱,Viewer
2,HR현황,미래혁신본부,디지털플랫폼팀,팀원,박미숙,Viewer
3,HR현황,미래혁신본부,미래전략팀,서무,김아현,Viewer
4,공수현황 접근그룹,개발영업본부,견적그룹(개발영업),그룹장,소문석,Viewer


### groups.create()

In [66]:
try:
    with server.auth.sign_in(tableau_auth):
        newgroup = TSC.GroupItem("마이호미")
        newgroup.minimum_site_role = TSC.UserItem.Roles.Viewer
        newgroup = server.groups.create(newgroup)
except:
    pass

### groups.add_user()

In [76]:
keyword = "호미"
num = "5634"

group_idx = all_groups[all_groups["group"].str.contains(keyword)].index[0]
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    group = groups[group_idx]
    
    user_idx = all_users[all_users["user_name"] == num].index[0]
    users, pagination_item = server.users.get(req_options=req_opts)
    user_id = users[user_idx].id
    
    server.groups.add_user(group, user_id)

## Users

### users.get(), users.populate_groups()

In [6]:
with server.auth.sign_in(tableau_auth):
    all_users, pagination_item = server.users.get(req_options=req_opts)
all_users = [(user.id, user.name, user.fullname, user.site_role) + empls[user.name] if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None) for user in all_users]
all_users = pd.DataFrame(all_users, columns=["user_id", "user_name", "user_fullname", "site_role", "hq", "dep", "job", "name"])
# all_users = all_users.sort_values(by=["hq", "dep", "job", "name", "site_role"])
# all_users = all_users.reset_index(drop=True)

In [7]:
all_users.to_csv("D:/디지털혁신팀/태블로/all_users.csv", index=False, encoding="euc-kr")
all_users.head()

,user_id,user_name,user_fullname,site_role,hq,dep,job,name
0,bdebde0b-b41d-4e9a-a6b7-423ab68bb16b,admin,admin,ServerAdministrator,None,None,None,None
1,a6e879b8-950e-4b05-9ff6-fe5565da5018,viewer,viewer(디지털플랫폼팀),Viewer,None,None,None,None
2,2c2ceaa8-9387-4edb-8ab2-38b17f686304,2912,김동진,Viewer,건설본부,건축PM,건축PM,김동진
3,c232b3be-3aac-4f4f-83fb-bc8a15dab963,3109,배영선,Viewer,건설본부,건축PM,건축PM,배영선
4,e17666e4-a736-40a7-badc-865a60ebd7f2,3139,서태철,Viewer,건설본부,광주학동4구역,소장,서태철


In [84]:
user_group = list()
with server.auth.sign_in(tableau_auth):
    users, pagination_item = server.users.get(req_options=req_opts)
    for user in users:
        pagination_item = server.users.populate_groups(user, req_options=req_opts)
        user_group.extend([empls[user.name] + (user.site_role, group.name) if user.name in empls.keys() else (None, None, None, user.name, user.site_role, group.name) for group in user.groups if group.name != "All Users"])
        
user_group = pd.DataFrame(user_group, columns=["hq", "dep", "job", "name", "site_role", "group"])
# user_group = user_group.sort_values(by=["hq", "dep", "job", "name", "site_role", "group"])
# user_group = user_group.reset_index(drop=True)

In [87]:
user_group

,hq,dep,job,name,site_role,group
0,None,None,None,admin,ServerAdministrator,디지털혁신팀
1,None,None,None,viewer,Viewer,디지털혁신팀
2,건설본부,건축PM,건축PM,김동진,Viewer,본사팀장/그룹장
3,건설본부,건축PM,건축PM,배영선,Viewer,본사팀장/그룹장
4,건설본부,외주구매팀,팀장,정상민,Viewer,본사팀장/그룹장
...,...,...,...,...,...,...
122,사장직속,안전경영실,팀원,고유경,ExplorerCanPublish,본부장 회의
123,개발영업본부,인프라개발팀,"도로,철도",박경신,Viewer,본부장 회의
124,미래혁신본부,디지털플랫폼팀,팀원,배인준,Viewer,마이호미
125,None,None,None,1101296,Viewer,분양일보


In [88]:
user_group[user_group["name"] == "김아현"]

,hq,dep,job,name,site_role,group
109,미래혁신본부,미래전략팀,서무,김아현,Viewer,HR현황
110,미래혁신본부,미래전략팀,서무,김아현,Viewer,분양일보
111,미래혁신본부,미래전략팀,서무,김아현,Viewer,본부장 회의


In [30]:
# user_group.to_csv("D:/디지털혁신팀/태블로/user_group.csv", index=False, encoding="euc-kr")
user_group.head()

,hq,dep,job,name,site_role,group
0,개발영업본부,개발부문,부문장,김태수,Viewer,본사팀장/그룹장
1,개발영업본부,개발영업본부,본부장,박희윤,Viewer,본부장 이상
2,개발영업본부,견적그룹(개발영업),그룹장,소문석,Viewer,본사팀장/그룹장
3,개발영업본부,도시개발1팀,팀원,임유빈,Viewer,"""본부장 회의"" 접근 가능 그룹"
4,개발영업본부,도시개발1팀,팀장,이현우,Viewer,"""분양일보"" 접근 가능 그룹"


### users.populate_workbooks()

In [31]:
with server.auth.sign_in(tableau_auth):
    user = server.users.get_by_id("515c7d10-953c-4bfc-af9a-e87cf34ccb85")
    page_n = server.users.populate_workbooks(user)
    for workbook in user.workbooks :
        print(workbook.name)

분양일보_DM 테스트 (2)
AS_210324
마이호미_210326
menu navi test3
CS 현황_test
분양일보_test_210326
진행 현장 실적
Test (현장현황)
menu navi test2
Test (협업플랫폼 사용량)
현장 조직도_210319
마이호미_210329
분양일보_test_월 단위_210329
TEST
CS 현황_6월 데이터 확인
원자재 가격_210202
마이호미_210223
마이호미_210325
분양일보_DM 테스트
설비그룹 조직도_210322
분양일보_모바일
현장 현황test
menu navi
동호표_210331
CS 현황
홈 화면_디자인 변경
마이호미_210322
분양일보_test_210329
협업플랫폼 사용량
본부장회의자료_test
진행 현장 실적
본부장회의자료
마이호미


### users.add()

In [115]:
with server.auth.sign_in(tableau_auth):
    name = "1101296"
    site_role = "Viewer"
    new_user = TSC.UserItem(name, site_role)
    
    new_user = server.users.add(new_user)   
    new_user.fullname = "황승환(디지털플랫폼팀)"
    new_user.email = "hwangsh@hdc-dvp.com"
    
    new_user = server.users.update(new_user, password=f"hdc{name}")

### user.update()

In [16]:
with server.auth.sign_in(tableau_auth):
    user = server.users.get_by_id("a6e879b8-950e-4b05-9ff6-fe5565da5018")
    user.fullname = "viewer(디지털플랫폼팀)"
    
    user_rev = server.users.update(user)

### user.remove()

In [114]:
with server.auth.sign_in(tableau_auth):
    server.users.remove("7103bcd9-9df5-4b0c-9e63-50698dd74392")

## Upload a Workbook

In [50]:
# filepath = "D:/디지털혁신팀/태블로/마이호미/마이호미_FINAL/마이호미.twbx"
# with server.auth.sign_in(tableau_auth):
#     wb = TSC.WorkbookItem(name="마이호미", project_id="44546321-94c8-4746-b0e5-0dfb3fa4ce5a")
#     wb = server.workbooks.publish(wb, filepath, "Overwrite")

### Sign Out

In [33]:
# server.auth.sign_out()